In [ ]:
import tensorflow as tf

### Layers
Las redes neuronales se organizan en capas de la forma

$h = f(w x + b) $

como vimos anteriormente. Sin embargo, es algo tedioso crear las variables `w` y `b` manualmente, para esto tensorflow nos brinda la funcion `tf.layers.dense`.

In [ ]:
x = tf.placeholder(tf.float64, [None, 4])
h = tf.layers.dense(x, 2, activation=tf.nn.softmax)

Aqui `f = tf.nn.softmax`, `x` un input de `4` dimensiones, y `y` es un tensor de `2` dimensiones. 

### Activation Functions
Algunas funciones de activacion utiles son:

* `None` si no pasamos funcion de activacion se utiliza identidad: $ f(x) = x$
* `tf.nn.sigmoid`: utilizada para clasificacion binaria: $ f(x) = \frac{1}{1 + exp(-x)} $
* `tf.nn.softmax`: utiliada para clasificacion multi-clase: $f(x) = \frac{exp(x)}{\sum_{j}exp(x_j)} $

### Loss Functions
#### mean squared errror
Si la funcion de la ultima capa es lineal, utilizamos `tf.nn.l2_loss` para medir el error

In [ ]:
#inputs
x = tf.placeholder(tf.float64, [None, 4])
y = tf.placeholder(tf.float64, [None, 2])

h = tf.layers.dense(x, 2)

loss = tf.nn.l2_loss(y - h)

##### sigmoid cross entropy error
Si la funcion de activacion de la ultima capa es `tf.nn.sigmoid`, utilizamos `tf.nn.sigmoid_cross_entropy_with_logits`. Notece que tiene que ser sobre lo logits ($ w x + b $) y no sobre `h`, aunque se podria calcular desde `h` existe una formula que es numericamente mas estable utilizando los logits.

In [ ]:
x = tf.placeholder(tf.float64, [None, 4])
logits = tf.layers.dense(x, 2)
h = tf.nn.sigmoid(logits)

loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=y)

##### softmax cross entropy error
Si la funcion de activacion de la ultima capa es `tf.nn.softmax`, utilizamos `tf.nn.softmax_cross_entropy_with_logits`. Notece que tiene que ser sobre lo logits ($ w x + b $) y no sobre `h`, aunque se podria calcular desde `h` existe una formula que es numericamente mas estable utilizando los logits.

In [ ]:
x = tf.placeholder(tf.float64, [None, 4])
logits = tf.layers.dense(x, 2)
h = tf.nn.softmax(logits)

loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y)

### Optimizers
Los optimizers calculan los gradientes del `loss` con respecto a las `Variables` de la red y generan una rutina interna para actualizar estas a fin de minimizar el `loss`. Algunos optimizadores populares son:

* `tf.train.GradientDescentOptimizer`: algortimo de gradiente descent puro.
* `tf.train.AdamOptimizer`: https://arxiv.org/abs/1412.6980
* `tf.train.AdadeltaOptimizer`: https://arxiv.org/abs/1212.5701
* `tf.train.RMSPropOptimizer`: no publicado, creado por G. Hinton.
* `tf.train.AdagradOptimizer`: http://jmlr.org/papers/v12/duchi11a.html

Para mas info de los optimizadores, les recomiendo este excellente blog: http://sebastianruder.com/optimizing-gradient-descent/index.html. Todos los optimizadores reciben como parametro el `learning_rate`. Ejemplo

In [ ]:
trainer = tf.train.AdamOptimizer(0.01)

Para crear la rutina del gradiente descent debemos utilizar el metodo `mimize(loss)` del trainer, esto nos devuelve un tensor que ejecuta la operacion de actualizar las variables.

In [ ]:
loss = tf.reduce_mean(loss)
update = trainer.minimize(loss)

### Gradiente Descent
Para realizar gradient descent solo necesitamos correr el tensor `update` multiples veces pasandole los valores de **x** y **y** del dataset en el `feed_dict`. El siguiente ejemplo ilustra como realizar el procedimiento pero saca error debido a que no tenemos definidas las variables necesarias

In [ ]:
import plotly.offline as py
from plotly import graph_objs as go
import numpy as np
py.init_notebook_mode()

n = 50

x_data = np.random.uniform(low=0.0, high=100.0, size=(n, 4))
w = np.asarray([[-0.2, 0.4, -0.7, 0.9],[0.13, 1.2, -1.7, 2.3]]).T
y_data = np.dot(x_data, w) + np.random.normal(loc=0.0, scale=25.0, size=(n, 2))

print(y_data.shape)

#normalizar data
x_data = (x_data - np.mean(x_data)) / np.average(x_data)
y_data = (y_data - np.mean(y_data)) / np.average(y_data)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(1000):
        _, _loss = sess.run([update, loss], feed_dict={x: x_data, y: y_data})
        print(_loss)